In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from exp_certify_multiclass import run

In [3]:
seed = 0

data_params = dict(
    dataset = "cora",
    learning_setting = "transductive", # or "transdructive"
    specification = dict(
        n_per_class = 10,
        fraction_test = 0.01,
        data_dir = "./data",
        make_undirected = True,
        binary_attr = False,
        balance_test = True,
    )
)

model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 1,
    pred_method = "svm",
    activation = "relu"
)

certificate_params = dict(
    n_adversarial = 1, # number adversarial nodes
    perturbation_model = "linf",
    delta = 0.01 # l0: local budget = delta * feature_dim
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "cpu",
    dtype = torch.float64,
    allow_tf32 = False
)

In [7]:
# MIP foucs = 2
#model_params["cache_size"] = 10000
data_params["dataset"] = "cora"
other_params["device"] = "0"
other_params["debug"] = False
certificate_params["n_adversarial"] = 270
certificate_params["delta"] = 0.1
certificate_params["delta_absolute"] = False
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "l2"
certificate_params["attack_nodes"] = "test"
certificate_params["MIPFocus"] = 2
certificate_params["LogToConsole"] = 0
certificate_params["OutputFlag"] = 0
other_params["path_gurobi_license"] = "/ceph/ssd/staff/gosl/app/gurobi.lic"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.001
model_params["solver"] = "qplayer_one_vs_all"
model_params["alpha_tol"] = 1e-4
model_params["bias"] = False
data_params["learning_setting"] = "transductive"
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-04-26 18:52:04 (INFO): Starting experiment exp_certify_multiclass with configuration:
2024-04-26 18:52:04 (INFO): data_params: {'dataset': 'cora', 'learning_setting': 'transductive', 'specification': {'n_per_class': 10, 'fraction_test': 0.01, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True}}
2024-04-26 18:52:04 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.001, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer_one_vs_all', 'alpha_tol': 0.0001, 'bias': False}
2024-04-26 18:52:04 (INFO): certification_params: {'n_adversarial': 270, 'perturbation_model': 'l2', 'delta': 0.1, 'delta_absolute': False, 'method': 'XXT', 'attack_nodes': 'test', 'MIPFocus': 2, 'LogToConsole': 0, 'OutputFlag': 0}
2024-04-26 18:52:04 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-04-26 18:52:04 (INFO): other_params: {'device': '0', 'dtype': torch.float64, 'allow_tf32': False

Class 0: 140 alphas found: ['0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0.0010', '0

2024-04-26 18:52:07 (INFO): Test accuracy: 0.6776556968688965
2024-04-26 18:52:07 (INFO): Test accuracy ALL UNLABELED: 0.6736760139465332
2024-04-26 18:52:07 (INFO): Train accuracy: 0.8785714507102966
2024-04-26 18:52:07 (INFO): Accuracy_lb_test: 0.6776556968688965
2024-04-26 18:52:07 (INFO): Accuracy_ub_test: 0.6776556968688965
2024-04-26 18:52:07 (INFO): Accuracy_lb_trn: 0.8785714507102966
2024-04-26 18:52:07 (INFO): Accuracy_ub_trn: 0.8785714507102966
2024-04-26 18:52:07 (INFO): Certified accuracy (evasion): 0.1318681389093399
2024-04-26 18:52:07 (INFO): Certified accuracy (evasion, trivial): 0.23443223443223443
2024-04-26 18:52:07 (INFO): Certified unrobustness (evasion): 0.0


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


2024-04-26 18:52:09 (INFO): Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


...

Set parameter IntFeasTol to value 0.0001


2024-04-26 18:53:29 (INFO): Set parameter IntFeasTol to value 0.0001


Loaded user MIP start with objective -0.339561



2024-04-26 18:53:29 (INFO): Optimization status: 2
2024-04-26 18:53:29 (INFO): Original -0.33956, Opt objective -0.37711, Opt bound: -0.37711, stop_obj: None


KeyboardInterrupt: 

In [10]:
# MIP foucs = 2
#model_params["cache_size"] = 10000
data_params["dataset"] = "cora_ml_cont"
other_params["device"] = "0"
other_params["debug"] = False
certificate_params["n_adversarial"] = 270
certificate_params["delta"] = 0.01
certificate_params["delta_absolute"] = False
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "l2"
certificate_params["attack_nodes"] = "test"
certificate_params["MIPFocus"] = 2
certificate_params["LogToConsole"] = 1
certificate_params["OutputFlag"] = 1
other_params["path_gurobi_license"] = "/ceph/ssd/staff/gosl/app/gurobi.lic"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.005
model_params["solver"] = "qplayer_one_vs_all"
model_params["alpha_tol"] = 1e-4
model_params["bias"] = False
data_params["learning_setting"] = "transductive"
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-04-25 21:56:42 (INFO): Starting experiment exp_certify_multiclass with configuration:
2024-04-25 21:56:42 (INFO): data_params: {'dataset': 'cora_ml_cont', 'learning_setting': 'transductive', 'specification': {'n_per_class': 10, 'fraction_test': 0.01, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True}}
2024-04-25 21:56:42 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.005, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer_one_vs_all', 'alpha_tol': 0.0001, 'bias': False}
2024-04-25 21:56:42 (INFO): certification_params: {'n_adversarial': 270, 'perturbation_model': 'l2', 'delta': 0.01, 'delta_absolute': False, 'method': 'XXT', 'attack_nodes': 'test', 'MIPFocus': 2, 'LogToConsole': 1, 'OutputFlag': 1}
2024-04-25 21:56:42 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-04-25 21:56:42 (INFO): other_params: {'device': '0', 'dtype': torch.float64, 'allow_tf3

Class 0: 53 alphas found: ['0.0000', '0.0050', '0.0050', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0007', '0.0030', '0.0000', '0.0050', '0.0000', '0.0000', '0.0000', '0.0025', '0.0000', '0.0000', '0.0000', '0.0000', '0.0050', '0.0050', '0.0050', '0.0000', '0.0000', '0.0000', '0.0000', '0.0050', '0.0000', '0.0013', '0.0050', '0.0000', '0.0000', '0.0000', '0.0050', '0.0000', '0.0000', '0.0000', '0.0050', '0.0000', '0.0000', '0.0050', '0.0000', '0.0050', '0.0050', '0.0000', '0.0000', '0.0050', '0.0002', '0.0000', '0.0000', '0.0050', '0.0000', '0.0000', '0.0043', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0050', '0.0001', '0.0000', '0.0000', '0.0000', '0.0050', '-0.0000', '0.0050', '0.0050', '0.0038', '0.0050', '0.0000', '0.0003', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0026', '0.0000', '0.0050', '0.0000', '0.0000', '0.0050', '0.0000', '0.0000', '0.0050', '0.0000', '0.0010', '0.0000', '0.0050', '0.0000', '0.0000', '0

2024-04-25 21:56:48 (INFO): Test accuracy: 0.7960526347160339
2024-04-25 21:56:48 (INFO): Test accuracy ALL UNLABELED: 0.7695271968841553
2024-04-25 21:56:48 (INFO): Train accuracy: 0.949999988079071
2024-04-25 21:56:48 (INFO): Accuracy_lb_test: 0.7894737124443054
2024-04-25 21:56:48 (INFO): Accuracy_ub_test: 0.7927631735801697
2024-04-25 21:56:48 (INFO): Accuracy_lb_trn: 0.949999988079071
2024-04-25 21:56:48 (INFO): Accuracy_ub_trn: 0.9428571462631226
2024-04-25 21:56:48 (INFO): Certified accuracy (evasion): 0.5493420958518982
2024-04-25 21:56:48 (INFO): Certified accuracy (evasion, trivial): 0.18421052631578946
2024-04-25 21:56:48 (INFO): Certified unrobustness (evasion): 0.0


AssertionError: 

In [5]:
# MIP foucs = 2
#model_params["cache_size"] = 10000
other_params["device"] = "cpu"
other_params["debug"] = False
certificate_params["n_adversarial"] = 270
certificate_params["delta"] = 0.01
certificate_params["delta_absolute"] = False
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "l2"
certificate_params["attack_nodes"] = "test"
certificate_params["MIPFocus"] = 2
certificate_params["LogToConsole"] = 1
certificate_params["OutputFlag"] = 1
other_params["path_gurobi_license"] = "/ceph/ssd/staff/gosl/app/gurobi.lic"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.0055
model_params["solver"] = "qplayer_one_vs_all"
model_params["alpha_tol"] = 1e-4
model_params["bias"] = False
data_params["learning_setting"] = "transductive"
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-04-25 18:54:01 (INFO): Starting experiment exp_certify_multiclass with configuration:
2024-04-25 18:54:01 (INFO): data_params: {'dataset': 'cora', 'learning_setting': 'transductive', 'specification': {'n_per_class': 10, 'fraction_test': 0.01, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True}}
2024-04-25 18:54:01 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.0055, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer_one_vs_all', 'alpha_tol': 0.0001, 'bias': False}
2024-04-25 18:54:01 (INFO): certification_params: {'n_adversarial': 270, 'perturbation_model': 'l2', 'delta': 0.01, 'delta_absolute': False, 'method': 'XXT', 'attack_nodes': 'test', 'MIPFocus': 2, 'LogToConsole': 1, 'OutputFlag': 1}
2024-04-25 18:54:01 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-04-25 18:54:01 (INFO): other_params: {'device': 'cpu', 'dtype': torch.float64, 'allow_tf32': F

2024-04-25 18:54:02 (INFO): number of samples
 - labeled: 70 
 - val: 70 
 - test: 273 
 - unlabeled: 2295
2024-04-25 18:54:03 (INFO): Delta: 0.01


Class 0: 108 alphas found: ['-0.0000', '0.0055', '0.0055', '0.0008', '0.0055', '0.0055', '0.0037', '0.0055', '0.0055', '0.0055', '0.0055', '-0.0000', '-0.0000', '0.0000', '0.0055', '0.0055', '0.0055', '0.0055', '0.0055', '-0.0000', '0.0055', '0.0055', '0.0055', '-0.0000', '0.0055', '-0.0000', '-0.0000', '0.0055', '0.0055', '0.0055', '-0.0000', '0.0055', '-0.0000', '0.0055', '0.0055', '0.0055', '-0.0000', '-0.0000', '0.0017', '0.0055', '0.0031', '0.0043', '0.0029', '0.0055', '0.0055', '0.0055', '0.0055', '0.0055', '0.0055', '0.0039', '0.0055', '0.0055', '-0.0000', '0.0055', '0.0043', '0.0027', '0.0055', '0.0055', '-0.0000', '0.0055', '0.0055', '0.0055', '-0.0000', '0.0002', '0.0055', '-0.0000', '0.0055', '0.0055', '0.0055', '0.0055', '0.0055', '0.0055', '-0.0000', '0.0055', '-0.0000', '-0.0000', '0.0055', '0.0055', '0.0055', '0.0055', '0.0055', '0.0055', '-0.0000', '0.0055', '0.0055', '0.0055', '-0.0000', '0.0055', '0.0055', '-0.0000', '0.0055', '0.0055', '-0.0000', '0.0024', '0.0055', 

2024-04-25 18:54:06 (INFO): Test accuracy: 0.7948718070983887
2024-04-25 18:54:06 (INFO): Test accuracy ALL UNLABELED: 0.7788162231445312
2024-04-25 18:54:06 (INFO): Train accuracy: 0.949999988079071
2024-04-25 18:54:06 (INFO): Accuracy_lb_test: 0.7948718070983887
2024-04-25 18:54:06 (INFO): Accuracy_ub_test: 0.791208803653717
2024-04-25 18:54:06 (INFO): Accuracy_lb_trn: 0.949999988079071
2024-04-25 18:54:06 (INFO): Accuracy_ub_trn: 0.949999988079071
2024-04-25 18:54:06 (INFO): Certified accuracy (evasion): 0.831501841545105
2024-04-25 18:54:06 (INFO): Certified accuracy (evasion, trivial): 0.23443223443223443
2024-04-25 18:54:06 (INFO): Certified unrobustness (evasion): 0.0


Set parameter Username


...

2024-04-25 18:55:02 (INFO): 


     0     0   -1.17523    0  111   -0.88431   -1.17523  32.9%     -    7s


2024-04-25 18:55:02 (INFO):      0     0   -1.17523    0  111   -0.88431   -1.17523  32.9%     -    7s


     0     1   -1.16725    0  110   -0.88431   -1.16725  32.0%     -   12s


2024-04-25 18:55:07 (INFO):      0     1   -1.16725    0  110   -0.88431   -1.16725  32.0%     -   12s


In [5]:
# MIP foucs = 2
#model_params["cache_size"] = 10000
other_params["device"] = "cpu"
other_params["debug"] = False
certificate_params["n_adversarial"] = 270
certificate_params["delta"] = 0.01
certificate_params["delta_absolute"] = False
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "l2"
certificate_params["attack_nodes"] = "test"
certificate_params["MIPFocus"] = 2
certificate_params["LogToConsole"] = 1
certificate_params["OutputFlag"] = 1
other_params["path_gurobi_license"] = "/ceph/ssd/staff/gosl/app/gurobi.lic"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.0075
model_params["solver"] = "qplayer_one_vs_all"
model_params["alpha_tol"] = 1e-4
model_params["bias"] = False
data_params["learning_setting"] = "transductive"
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-04-25 18:50:47 (INFO): Starting experiment exp_certify_multiclass with configuration:
2024-04-25 18:50:47 (INFO): data_params: {'dataset': 'cora', 'learning_setting': 'transductive', 'specification': {'n_per_class': 10, 'fraction_test': 0.01, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True}}
2024-04-25 18:50:47 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.0075, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer_one_vs_all', 'alpha_tol': 0.0001, 'bias': False}
2024-04-25 18:50:47 (INFO): certification_params: {'n_adversarial': 270, 'perturbation_model': 'l2', 'delta': 0.01, 'delta_absolute': False, 'method': 'XXT', 'attack_nodes': 'test', 'MIPFocus': 2, 'LogToConsole': 1, 'OutputFlag': 1}
2024-04-25 18:50:47 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-04-25 18:50:47 (INFO): other_params: {'device': 'cpu', 'dtype': torch.float64, 'allow_tf32': F

2024-04-25 18:50:47 (INFO): number of samples
 - labeled: 70 
 - val: 70 
 - test: 273 
 - unlabeled: 2295
2024-04-25 18:50:51 (INFO): Delta: 0.01


Class 0: 95 alphas found: ['-0.0000', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0075', '0.0020', '0.0075', '0.0075', '0.0075', '0.0075', '-0.0000', '-0.0000', '-0.0000', '0.0075', '0.0075', '0.0075', '0.0075', '0.0038', '-0.0000', '0.0075', '0.0043', '0.0075', '-0.0000', '0.0075', '-0.0000', '-0.0000', '0.0075', '0.0075', '0.0075', '-0.0000', '0.0075', '-0.0000', '0.0075', '0.0075', '0.0075', '-0.0000', '-0.0000', '-0.0000', '0.0075', '-0.0000', '0.0012', '-0.0000', '0.0075', '0.0075', '0.0075', '0.0075', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0000', '-0.0000', '0.0075', '0.0075', '-0.0000', '0.0031', '0.0075', '0.0075', '-0.0000', '-0.0000', '0.0018', '-0.0000', '0.0075', '0.0020', '0.0075', '0.0075', '0.0075', '0.0075', '-0.0000', '0.0075', '-0.0000', '-0.0000', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0075', '0.0045', '-0.0000', '0.0075', '0.0056', '0.0075', '-0.0000', '-0.0000', '0.0075', '-0.0000', '0.0075', '0.0075', '-0.0000', '-0.0000', 

2024-04-25 18:50:58 (INFO): Test accuracy: 0.7875458002090454
2024-04-25 18:50:58 (INFO): Test accuracy ALL UNLABELED: 0.7776479721069336
2024-04-25 18:50:58 (INFO): Train accuracy: 0.949999988079071
2024-04-25 18:50:58 (INFO): Accuracy_lb_test: 0.791208803653717
2024-04-25 18:50:58 (INFO): Accuracy_ub_test: 0.7875458002090454
2024-04-25 18:50:58 (INFO): Accuracy_lb_trn: 0.949999988079071
2024-04-25 18:50:58 (INFO): Accuracy_ub_trn: 0.949999988079071
2024-04-25 18:50:58 (INFO): Certified accuracy (evasion): 0.8241758346557617
2024-04-25 18:50:58 (INFO): Certified accuracy (evasion, trivial): 0.23443223443223443
2024-04-25 18:50:58 (INFO): Certified unrobustness (evasion): 0.0


Set parameter Username


...

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


2024-04-25 18:53:11 (INFO):  Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


2024-04-25 18:53:11 (INFO): 


     0     0   -1.43085    0  120   -0.87296   -1.43085  63.9%     -    6s


2024-04-25 18:53:11 (INFO):      0     0   -1.43085    0  120   -0.87296   -1.43085  63.9%     -    6s


In [5]:
# MIP foucs = 0
#model_params["cache_size"] = 10000
other_params["device"] = "cpu"
other_params["debug"] = False
certificate_params["n_adversarial"] = 270
certificate_params["delta"] = 0.01
certificate_params["delta_absolute"] = False
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "l2"
certificate_params["attack_nodes"] = "test"
certificate_params["MIPFocus"] = 0
certificate_params["LogToConsole"] = 1
certificate_params["OutputFlag"] = 1
other_params["path_gurobi_license"] = "/ceph/ssd/staff/gosl/app/gurobi.lic"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.0075
model_params["solver"] = "qplayer_one_vs_all"
model_params["alpha_tol"] = 1e-4
model_params["bias"] = False
data_params["learning_setting"] = "transductive"
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-04-25 18:47:40 (INFO): Starting experiment exp_certify_multiclass with configuration:
2024-04-25 18:47:40 (INFO): data_params: {'dataset': 'cora', 'learning_setting': 'transductive', 'specification': {'n_per_class': 10, 'fraction_test': 0.01, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True}}
2024-04-25 18:47:40 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.0075, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer_one_vs_all', 'alpha_tol': 0.0001, 'bias': False}
2024-04-25 18:47:40 (INFO): certification_params: {'n_adversarial': 270, 'perturbation_model': 'l2', 'delta': 0.01, 'delta_absolute': False, 'method': 'XXT', 'attack_nodes': 'test', 'MIPFocus': 0, 'LogToConsole': 1, 'OutputFlag': 1}
2024-04-25 18:47:40 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-04-25 18:47:40 (INFO): other_params: {'device': 'cpu', 'dtype': torch.float64, 'allow_tf32': F

2024-04-25 18:47:40 (INFO): number of samples
 - labeled: 70 
 - val: 70 
 - test: 273 
 - unlabeled: 2295
2024-04-25 18:47:45 (INFO): Delta: 0.01


Class 0: 95 alphas found: ['-0.0000', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0075', '0.0020', '0.0075', '0.0075', '0.0075', '0.0075', '-0.0000', '-0.0000', '-0.0000', '0.0075', '0.0075', '0.0075', '0.0075', '0.0038', '-0.0000', '0.0075', '0.0043', '0.0075', '-0.0000', '0.0075', '-0.0000', '-0.0000', '0.0075', '0.0075', '0.0075', '-0.0000', '0.0075', '-0.0000', '0.0075', '0.0075', '0.0075', '-0.0000', '-0.0000', '-0.0000', '0.0075', '-0.0000', '0.0012', '-0.0000', '0.0075', '0.0075', '0.0075', '0.0075', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0000', '-0.0000', '0.0075', '0.0075', '-0.0000', '0.0031', '0.0075', '0.0075', '-0.0000', '-0.0000', '0.0018', '-0.0000', '0.0075', '0.0020', '0.0075', '0.0075', '0.0075', '0.0075', '-0.0000', '0.0075', '-0.0000', '-0.0000', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0075', '0.0045', '-0.0000', '0.0075', '0.0056', '0.0075', '-0.0000', '-0.0000', '0.0075', '-0.0000', '0.0075', '0.0075', '-0.0000', '-0.0000', 

2024-04-25 18:48:00 (INFO): Test accuracy: 0.7875458002090454
2024-04-25 18:48:00 (INFO): Test accuracy ALL UNLABELED: 0.7776479721069336
2024-04-25 18:48:00 (INFO): Train accuracy: 0.949999988079071
2024-04-25 18:48:00 (INFO): Accuracy_lb_test: 0.791208803653717
2024-04-25 18:48:00 (INFO): Accuracy_ub_test: 0.7875458002090454
2024-04-25 18:48:00 (INFO): Accuracy_lb_trn: 0.949999988079071
2024-04-25 18:48:00 (INFO): Accuracy_ub_trn: 0.949999988079071
2024-04-25 18:48:00 (INFO): Certified accuracy (evasion): 0.8241758346557617
2024-04-25 18:48:00 (INFO): Certified accuracy (evasion, trivial): 0.23443223443223443
2024-04-25 18:48:00 (INFO): Certified unrobustness (evasion): 0.0


Set parameter Username


...

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


2024-04-25 18:50:28 (INFO):  Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


2024-04-25 18:50:28 (INFO): 


     0     0   -1.43085    0  120   -0.87296   -1.43085  63.9%     -    5s


2024-04-25 18:50:28 (INFO):      0     0   -1.43085    0  120   -0.87296   -1.43085  63.9%     -    5s


In [8]:
# MIP foucs = 3
#model_params["cache_size"] = 10000
other_params["device"] = "cpu"
other_params["debug"] = False
certificate_params["n_adversarial"] = 270
certificate_params["delta"] = 0.01
certificate_params["delta_absolute"] = False
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "l2"
certificate_params["attack_nodes"] = "test"
#certificate_params["MIPGap"] = 0
other_params["path_gurobi_license"] = "/ceph/ssd/staff/gosl/app/gurobi.lic"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.0075
model_params["solver"] = "qplayer_one_vs_all"
model_params["alpha_tol"] = 1e-4
model_params["bias"] = False
data_params["learning_setting"] = "transductive"
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-04-25 18:27:24 (INFO): Starting experiment exp_certify_multiclass with configuration:
2024-04-25 18:27:24 (INFO): data_params: {'dataset': 'cora', 'learning_setting': 'transductive', 'specification': {'n_per_class': 10, 'fraction_test': 0.01, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True}}
2024-04-25 18:27:24 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.0075, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer_one_vs_all', 'alpha_tol': 0.0001, 'bias': False}
2024-04-25 18:27:24 (INFO): certification_params: {'n_adversarial': 270, 'perturbation_model': 'l2', 'delta': 0.01, 'delta_absolute': False, 'method': 'XXT', 'attack_nodes': 'test'}
2024-04-25 18:27:24 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-04-25 18:27:24 (INFO): other_params: {'device': 'cpu', 'dtype': torch.float64, 'allow_tf32': False, 'debug': False, 'path_gurobi_license': '/ceph

Class 0: 95 alphas found: ['-0.0000', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0075', '0.0020', '0.0075', '0.0075', '0.0075', '0.0075', '-0.0000', '-0.0000', '-0.0000', '0.0075', '0.0075', '0.0075', '0.0075', '0.0038', '-0.0000', '0.0075', '0.0043', '0.0075', '-0.0000', '0.0075', '-0.0000', '-0.0000', '0.0075', '0.0075', '0.0075', '-0.0000', '0.0075', '-0.0000', '0.0075', '0.0075', '0.0075', '-0.0000', '-0.0000', '-0.0000', '0.0075', '-0.0000', '0.0012', '-0.0000', '0.0075', '0.0075', '0.0075', '0.0075', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0000', '-0.0000', '0.0075', '0.0075', '-0.0000', '0.0031', '0.0075', '0.0075', '-0.0000', '-0.0000', '0.0018', '-0.0000', '0.0075', '0.0020', '0.0075', '0.0075', '0.0075', '0.0075', '-0.0000', '0.0075', '-0.0000', '-0.0000', '0.0075', '0.0075', '-0.0000', '0.0075', '0.0075', '0.0045', '-0.0000', '0.0075', '0.0056', '0.0075', '-0.0000', '-0.0000', '0.0075', '-0.0000', '0.0075', '0.0075', '-0.0000', '-0.0000', 

2024-04-25 18:27:47 (INFO): Test accuracy: 0.7875458002090454
2024-04-25 18:27:47 (INFO): Test accuracy ALL UNLABELED: 0.7776479721069336
2024-04-25 18:27:47 (INFO): Train accuracy: 0.949999988079071
2024-04-25 18:27:47 (INFO): Accuracy_lb_test: 0.791208803653717
2024-04-25 18:27:47 (INFO): Accuracy_ub_test: 0.7875458002090454
2024-04-25 18:27:47 (INFO): Accuracy_lb_trn: 0.949999988079071
2024-04-25 18:27:47 (INFO): Accuracy_ub_trn: 0.949999988079071
2024-04-25 18:27:47 (INFO): Certified accuracy (evasion): 0.8241758346557617
2024-04-25 18:27:47 (INFO): Certified accuracy (evasion, trivial): 0.23443223443223443
2024-04-25 18:27:47 (INFO): Certified unrobustness (evasion): 0.0


Set parameter Username


2024-04-25 18:27:47 (INFO): Set parameter Username


...

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


2024-04-25 18:46:09 (INFO):  Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


2024-04-25 18:46:09 (INFO): 


     0     0   -1.43085    0  120   -0.87296   -1.43085  63.9%     -    7s


2024-04-25 18:46:09 (INFO):      0     0   -1.43085    0  120   -0.87296   -1.43085  63.9%     -    7s


In [43]:
# MIP foucs = 3, seed 4, new big M_u, M_v
#model_params["cache_size"] = 10000
other_params["device"] = "cpu"
other_params["debug"] = False
certificate_params["n_adversarial"] = 270
certificate_params["delta"] = 0.01
certificate_params["delta_absolute"] = False
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "l2"
certificate_params["attack_nodes"] = "test"
other_params["path_gurobi_license"] = "/ceph/ssd/staff/gosl/app/gurobi.lic"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.002
model_params["solver"] = "qplayer_one_vs_all"
model_params["alpha_tol"] = 1e-4
model_params["bias"] = False
data_params["learning_setting"] = "transductive"
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-04-25 17:15:39 (INFO): Starting experiment exp_certify_multiclass with configuration:
2024-04-25 17:15:39 (INFO): data_params: {'dataset': 'cora', 'learning_setting': 'transductive', 'specification': {'n_per_class': 20, 'fraction_test': 0.01, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True}}
2024-04-25 17:15:39 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.002, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer_one_vs_all', 'alpha_tol': 0.0001, 'bias': False}
2024-04-25 17:15:39 (INFO): certification_params: {'n_adversarial': 270, 'perturbation_model': 'l2', 'delta': 0.01, 'delta_absolute': False, 'method': 'XXT', 'attack_nodes': 'test'}
2024-04-25 17:15:39 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-04-25 17:15:39 (INFO): other_params: {'device': 'cpu', 'dtype': torch.float64, 'allow_tf32': False, 'debug': False, 'path_gurobi_license': '/ceph/

Class 0: 241 alphas found: ['0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '-0.0000', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '-0.0000', '0.0020', '0.0020', '0.0020', '0.0020', '-0.0000', '0.0010', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0006', '0.0009', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '-0.0000', '0.0020', '0.0000', '0.0020', '0.0020', '0.0020', '0.0003', '0.0020', '0.0020', '0.0020', '0.0020', '-0.0000', '0.0020', '0.0020', '-0.0000', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '-0.0000', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '-0.0000', '0.0009', '0.0020', '0.0020', '0.0020', '0.0020', '0.0020', '-0.0000', '0.0020', '0.0020', '0.0020', '0.0020', '-0.0000', '0.0020', '0.0020', '-0.0000', '0.0020', '

2024-04-25 17:15:43 (INFO): Delta: 0.01
2024-04-25 17:15:58 (INFO): Test accuracy: 0.8351648449897766
2024-04-25 17:15:58 (INFO): Test accuracy ALL UNLABELED: 0.7940691709518433
2024-04-25 17:15:58 (INFO): Train accuracy: 0.9071428775787354
2024-04-25 17:15:58 (INFO): Accuracy_lb_test: 0.831501841545105
2024-04-25 17:15:58 (INFO): Accuracy_ub_test: 0.8351648449897766
2024-04-25 17:15:58 (INFO): Accuracy_lb_trn: 0.9071428775787354
2024-04-25 17:15:58 (INFO): Accuracy_ub_trn: 0.9071428775787354
2024-04-25 17:15:58 (INFO): Certified accuracy (evasion): 0.8351648449897766
2024-04-25 17:15:58 (INFO): Certified accuracy (evasion, trivial): 0.23809523809523808
2024-04-25 17:15:58 (INFO): Certified unrobustness (evasion): 0.0


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


2024-04-25 17:16:02 (INFO): Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


...

Best objective -7.462174618530e-01, best bound 2.309158521556e-01, gap 130.9448%


2024-04-25 17:44:49 (INFO): Best objective -7.462174618530e-01, best bound 2.309158521556e-01, gap 130.9448%


2024-04-25 17:44:49 (INFO): 


User-callback calls 1269, time in user-callback 0.01 sec


2024-04-25 17:44:49 (INFO): User-callback calls 1269, time in user-callback 0.01 sec
2024-04-25 17:44:49 (INFO): Optimization status: 15
2024-04-25 17:44:49 (INFO): Original -0.74622, Opt objective -0.74622, stop_obj: -0.8012007335889882
2024-04-25 17:44:49 (INFO): Robust count 0 out of 2
